In [5]:
import pandas as pd
import numpy as np
from scipy import stats

from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.metrics import r2_score , mean_squared_error
from sklearn.metrics import accuracy_score ,mean_absolute_error

In [6]:
import warnings
warnings.filterwarnings("ignore")

# Load the data

In [7]:
x = pd.read_csv("../trgc10000.csv")
x = x.iloc[:,1:]
y = pd.read_csv("../srlbininfo_10000",sep ="\t",header =None )
y = pd.DataFrame(y.iloc[:,1])
y.columns = ["GC"]

df = pd.concat([y,x],axis =1)

df = df[df.GC < 20]             # !  Ignore values more than 25
y = pd.DataFrame(df.loc[:,"GC"])

x = df.iloc[:,1:]  #    ! * remove 8000 for original

#Caution X Column up to 8000 above that outlier and y.GC Less than 

In [8]:
display(x.head(2))
display(y.head(2))

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V10323,V10324,V10325,V10326,V10327,V10328,V10329,V10330,V10331,V10332
0,0.879025,0.687329,0.983194,0.970820,0.900728,0.735702,0.960757,1.074772,1.597032,1.156174,...,0,0,0,0,0,0,0,0,0,0
1,0.869801,0.620215,1.008849,0.926203,0.884105,0.884811,1.020117,0.987255,1.335856,1.170143,...,0,0,0,0,0,0,0,0,0,0


,GC
0,19.052706
1,17.772070


In [115]:
x_train= None
x_test = None
y_train = None
y_test = None
y_pred_ridge = None
clf_lasso = None ; y_pred_lasso = None

In [87]:
def train_test(size = 0.25 , random_state=50,shuffle = True):
    "size = train size and random state is how the data are mixed and shuffle in to do mixing or not  "
    from sklearn.model_selection import train_test_split
    global x_train,x_test,y_train, y_test
    x_train= None; x_test = None; y_train = None; y_test = None
    x_train,x_test,y_train, y_test = train_test_split(x,y,test_size= size,random_state = random_state , shuffle = shuffle)
    
    return x_train,x_test,y_train, y_test 

In [196]:
#train_test() # Calling The Function with default Parameter
#print(y_train.shape)
#display(y_train.head(2))
#print(x_train.shape)
#display(x_train.head())

##  Create a Function for Lasso

In [213]:
#Create a Lasso Model to automatically apply in the loop

def lasso( alpha=0.5, fit_intercept=True, normalize=False, precompute=False, copy_X=True, 
                      max_iter=1000, tol=0.0001, warm_start=False, positive=False,selection='cyclic'):
    
    "Just Pass Related values only"
    
    from sklearn.linear_model import Lasso
    from sklearn.model_selection import train_test_split
    global x_train,x_test,y_train, y_test , clf_lasso , y_pred_lasso
    
    clf_lasso = Lasso(alpha=alpha, fit_intercept=fit_intercept , normalize=normalize, precompute=precompute, copy_X=copy_X, 
                      max_iter=max_iter, tol=tol, warm_start=warm_start, positive=positive,selection=selection).fit(x_train,y_train)
    
    y_pred_lasso = pd.DataFrame(clf_lasso.predict(x_test))
    name = "Lasso"
    # *****************************************************************
    
    print(name +" Coefficient ", clf_lasso.coef_) 
    print(name + " Intercept " , clf_lasso.intercept_,"\n") ##
    print("****"*5+"Accuracy Test Model Fitting "+"****"*5)
    print(name + "Score for train data set :",clf_lasso.score(x_train,y_train))
    print(name + "Score for test data Set",clf_lasso.score(x_test,y_test))

    print(name + "Score for Predictecd data Set",clf_lasso.score(x_test,y_pred_lasso)) ##
    
    print("\n"+"****"*5+"R2 Score"+"****"*5)
    print(name + "R2 Score for test is = ", r2_score(y_test, y_pred_lasso)) ##
    
    print("\n"+"****"*5+"Mean Absolute Error"+"****"*5)
    print(name+ "Mean Absolute Error of Test = ", mean_absolute_error(y_test,y_pred_lasso)) ##
    print(name +"Mean Absolute Error on Train",mean_absolute_error(y_train,pd.DataFrame(clf_lasso.predict(x_train))))
    
    print("\n"+"****"*5+" Mean Squared Error"+"****"*5)
    print(name + "Mean Squared Error of Test = ",mean_squared_error(y_test,y_pred_lasso)) ##
    print(name+"Mean Squared Error on Train = ",mean_squared_error(y_train,pd.DataFrame(clf_lasso.predict(x_train))))
    
    print("\n"+"****"*5+" Co Relation"+"****"*5)
    print(name + "Correlation Test Data ",stats.pearsonr(y_test["GC"],y_pred_lasso[0])[0])
    print(name + "Correlation Original Data ",stats.pearsonr(y_train["GC"],pd.DataFrame(clf_lasso.predict(x_train))[0])[0])
    
    return clf_lasso

In [200]:
clf_lasso

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

## Function For Ridge Regression

In [212]:
#Create a Model to automatically apply in the loop

def ridge( alpha = 0.01,copy_X=True,fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001):
    "Just Pass alpha value only"
    
    from sklearn.linear_model import Ridge
    from sklearn.model_selection import train_test_split
    global x_train,x_test,y_train, y_test , clf_ridge , y_pred_ridge
    
    clf_ridge = Ridge(alpha=alpha,copy_X=copy_X,fit_intercept=fit_intercept, max_iter=max_iter, normalize=normalize,
      random_state=random_state, solver=solver, tol=tol).fit(x_train,y_train)
    
    y_pred_ridge = pd.DataFrame(clf_ridge.predict(x_test))
    name = "Ridge"
    # *****************************************************************
    
    print(name +" Coefficient ", clf_ridge.coef_) 
    print(name + " Intercept " , clf_ridge.intercept_,"\n") ##
    print("****"*5+"Accuracy Test Model Fitting "+"****"*5)
    print(name + "Score for train data set :",clf_ridge.score(x_train,y_train))
    #Return the coefficient of determination R^2 of the prediction.
    
    print(name + "Score for test data Set",clf_ridge.score(x_test,y_test))

    print(name + "Score for Predictecd data Set",clf_ridge.score(x_test,y_pred_ridge)) ##
    
    print("\n"+"****"*5+"R2 Score"+"****"*5)
    print(name + "R2 Score for test is = ", r2_score(y_test, y_pred_ridge)) ##
    
    print("\n"+"****"*5+"Mean Absolute Error"+"****"*5)
    print(name+ "Mean Absolute Error of Test = ", mean_absolute_error(y_test,y_pred_ridge)) ##
    print(name +"Mean Absolute Error on Train",mean_absolute_error(y_train,pd.DataFrame(clf_ridge.predict(x_train))))
    
    print("\n"+"****"*5+" Mean Squared Error"+"****"*5)
    print(name + "Mean Squared Error of Test = ",mean_squared_error(y_test,y_pred_ridge)) ##
    print(name+"Mean Squared Error on Train = ",mean_squared_error(y_train,pd.DataFrame(clf_ridge.predict(x_train))))
    
    print("\n"+"****"*5+" Co Relation"+"****"*5)
    print(name + "Correlation Test Data ",stats.pearsonr(y_test["GC"],y_pred_ridge[0])[0])
    print(name + "Correlation Original Data ",stats.pearsonr(y_train["GC"],pd.DataFrame(clf_ridge.predict(x_train))[0])[0])
    
    return clf_ridge

### Call the Function in  with different Parameter

In [217]:
#We have 2 Function 
train_test(size = 0.25 , random_state=50,shuffle = True)
#***********************************************************
ridge(alpha=0.05,copy_X=True,fit_intercept=False, max_iter=1000, normalize=True)

Ridge Coefficient  [[0.42907626 0.41743016 0.37849042 ... 0.         0.         0.        ]]
Ridge Intercept  0.0 

********************Accuracy Test Model Fitting ********************
RidgeScore for train data set : 0.9990714572459918
RidgeScore for test data Set -0.6861394303892034
RidgeScore for Predictecd data Set 1.0

********************R2 Score********************
RidgeR2 Score for test is =  -0.6861394303892034

********************Mean Absolute Error********************
RidgeMean Absolute Error of Test =  4.129003272591964
RidgeMean Absolute Error on Train 0.09690898312721194

******************** Mean Squared Error********************
RidgeMean Squared Error of Test =  27.84812053102007
RidgeMean Squared Error on Train =  0.015251153274780252

******************** Co Relation********************
RidgeCorrelation Test Data  0.5196072823863953
RidgeCorrelation Original Data  0.9995480317853414


Ridge(alpha=0.05, copy_X=True, fit_intercept=False, max_iter=1000,
      normalize=True, random_state=None, solver='auto', tol=0.001)

### Lasso Regression Test

In [216]:
#We have 2 Function 
train_test(size=0.3,random_state=1000, shuffle=True)
lasso( alpha=0, fit_intercept=True, normalize=False, precompute=False, copy_X=True,max_iter=1000, tol=0.0001,warm_start=False, positive=False,selection='cyclic')

Lasso Coefficient  [ 0.37341145  0.86938701 -0.26948227 ...  0.          0.
  0.        ]
Lasso Intercept  [-1029.70893442] 

********************Accuracy Test Model Fitting ********************
LassoScore for train data set : 0.9999927742344025
LassoScore for test data Set -0.7272031082906196
LassoScore for Predictecd data Set 1.0

********************R2 Score********************
LassoR2 Score for test is =  -0.7272031082906196

********************Mean Absolute Error********************
LassoMean Absolute Error of Test =  4.127395370371444
LassoMean Absolute Error on Train 0.008613831591816463

******************** Mean Squared Error********************
LassoMean Squared Error of Test =  28.249646452682036
LassoMean Squared Error on Train =  0.00011910113412856906

******************** Co Relation********************
LassoCorrelation Test Data  0.5080182089908603
LassoCorrelation Original Data  0.9999963908704236


Lasso(alpha=0, copy_X=True, fit_intercept=True, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=None, selection='cyclic',
      tol=0.0001, warm_start=False)